In [1]:
import torch
import cv2
from segment_anything import sam_model_registry, SamPredictor
from segment_anything import SamAutomaticMaskGenerator_mod2 as SamAutomaticMaskGenerator
import numpy as np
import matplotlib.pyplot as plt
import glob
import torch
import functions as fnc
from importlib import reload
import gc
from skimage.measure import label, regionprops
from collections import Counter
from torchvision.ops.boxes import batched_nms
from sklearn.neighbors import KDTree
import matplotlib.colors as mcolors
import os
import time
import sys
import json
import pandas as pd
from contextlib import redirect_stdout

In [2]:
OutDIR='/DATA/vito/output/ran_synth_32_bw/ran_synth_32_bw_00_dw4_cp1024_3b/'
with open(OutDIR+'init_para.json', 'r') as json_file:
    init_para = json.load(json_file)
print('Loaded parameters from json')
print(init_para)

Loaded parameters from json
{'OutDIR': '/DATA/vito/output/ran_synth_32_bw/ran_synth_32_bw_00_dw4_cp1024_3b/', 'DataDIR': '/DATA/vito/data/', 'DatasetName': 'ran_synth_32_bw/img/*', 'fid': 0, 'crop_size': 1024, 'resample_factor': 1, 'point_per_side': 30, 'dilation_size': 15, 'b': 100, 'stability_t': 0.85, 'third_b_resample_factor': 0.08333333333333333, 'resolution(mm)': 0.2, 'expected_min_size(sqmm)': 0, 'min_radius': 0}


In [3]:
OutDIR=init_para.get('OutDIR')
DataDIR=init_para.get('DataDIR')
DSname=init_para.get('DatasetName')
fid=init_para.get('fid')
pps=init_para.get('point_per_side')
#window_step=init_para.get('window_step')
b=init_para.get('b')
stb_t=init_para.get('stability_t')
#defining clips
crop_size=init_para.get('crop_size')
resample_factor=init_para.get('resample_factor')
min_pixel=(init_para.get('expected_min_size(sqmm)')/(init_para.get('resolution(mm)')**2))*resample_factor
min_radi=init_para.get('min_radius')
print(f'Minimum expected size: {min_pixel} pixel')

Minimum expected size: 0.0 pixel


In [4]:

pre_para={'Resample': {'fxy':resample_factor},
        #'Gaussian': {'kernel size':3}
        #'CLAHE':{'clip limit':2}#,
        #'Resample': {'fxy':4},
        #'Buffering': {'crop size': crop_size}
        }
try:#attempt to load saved init_para
    with open(OutDIR+'pre_para.json', 'r') as json_file:
        init_para = json.load(json_file)
    print('Loaded preprocessing parameters from json')
    print(pre_para)
except:#use defined init_para
    print('Using preprocessing default')
    print(pre_para)

Loaded preprocessing parameters from json
{'Resample': {'fxy': 1}}


In [5]:
#Prep
#load image
image=fnc.load_image(DataDIR,DSname,fid)
print('Image size:', image.shape)
image=fnc.preprocessing_roulette(image, pre_para)

patches = fnc.get_image_patches(image, crop_size, 2*b)
print(f'Tiled into {len(patches)} patches')
patch_keys=patches.keys()
max_ij=np.max(np.array(list(patch_keys)),axis=0)

img_00.npy imported
Image size: (10000, 10000, 3)
Not resampling
Tiled into 144 patches


In [6]:
#setup SAM
MODEL_TYPE = "vit_h"
DEVICE, CHECKPOINT_PATH=fnc.set_sam(MODEL_TYPE,DataDIR)

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
sam.to(device=DEVICE)
mask_generator = SamAutomaticMaskGenerator(sam)

Currently running on GPU
Model vit_h


/DATA/vito/.venv/lib/python3.12/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)
